# CIS Bank File Load Info
Get database and FTP folder contents details related to CIS bank file load

In [36]:
import getpass
import time
import paramiko
import pyodbc
import pandas as pd
from IPython.display import Markdown

In [37]:
def run_query(database, username, password, sql, params=()):
    """Run SQL query and return results as pandas dataframe"""
    constring = "Driver={Oracle in Instantclient11_1};Dbq=%s;Uid=%s;Pwd=%s" % (database, username, password)
    conn = pyodbc.connect(constring)
    df = pd.read_sql_query(sql, conn, params=params)
    conn.close()
    return df

def run_ssh_command(ssh_host, ssh_user, ssh_password, ssh_command="pwd"):
    """Run command over SSH and return resulting stdout as string"""
    response = ""
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(ssh_host, port=22, username=ssh_user, password=ssh_password)
    stdin, stdout, stderr = client.exec_command(ssh_command)
    if stdout:
        response = stdout.readlines()
    client.close()
    return "".join(response)

### Database and ssh login details

In [38]:
# CIS Database
database = "E10DTCIS"
username = "train1"
password = getpass.getpass(database + " password: ")

# CIS Database Server
ssh_host = "10.223.60.123"
ssh_user = "batman"
ssh_password = getpass.getpass(ssh_user + " password:")

### Database Queries

In [39]:
tft_sql = "select * from tfr_bank_details"

# Copied from IDP Client Cash Receipts, FAM0R005 Modifications - Technical Specification v1.0.doc
batch_runs_sql = """
SELECT blog.*
     , berr.process_id
     , berr.error
  FROM (
       SELECT *
         FROM (
              SELECT blog_id
                   , date_to_run
                   , process_state
                FROM batch_logs
               WHERE module_id = 'FAM0R005'
               ORDER BY blog_id DESC
              )
        WHERE ROWNUM <= 10
       )
       blog
  LEFT OUTER JOIN batch_errors berr
    ON berr.blog_blog_id = blog.blog_id
 ORDER BY
       blog.blog_id desc
     , berr.process_id
"""

# Recnet info from bank_files table
bafi_sql = """
select * from bank_files
where bafi_id >
(select max(bafi_id)-20 from bank_files)
order by bafi_id desc
"""

# BANK_FILE_DET_RECS
bafi_det_recs = """
select * from bank_file_det_recs where bafi_bafi_id in 
(select max(bafi_id) from bank_files)
"""

# CCMS details extracted from last loaded file
ccms_payments = """
select * from tfr_ccms_bank_details
where cis_file_name =
    (select narrative from tfr_bank_details where cis_case = 'X')
"""

# Run the queries and display the results
queries = [batch_runs_sql, tft_sql, bafi_sql, bafi_det_recs, ccms_payments]
headings = ["Latest FAM0R005 Runs", "Bankfile Table", "Latest Files", "Bafi Det Recs", "CCMS Payments"]

display(Markdown(time.strftime("**%H:%M:%S %d/%m/%y**")))
for sql, heading in zip(queries, headings):
    df = run_query(database, username, password, sql)
    display(Markdown(heading))
    display(df)

**14:05:48 18/06/19**

Latest FAM0R005 Runs

,BLOG_ID,DATE_TO_RUN,PROCESS_STATE,PROCESS_ID,ERROR
0,1295045.0,2019-06-18 12:30:02,C,None,None
1,1291470.0,2018-09-25 13:11:34,C,None,None
2,1280436.0,2018-02-23 08:30:15,C,None,None
3,1280435.0,2018-02-23 08:30:14,C,None,None
4,1280434.0,2018-02-23 08:30:14,C,None,None
5,1280433.0,2018-02-23 08:30:10,C,None,None
6,1280432.0,2018-02-23 08:30:09,C,None,None
7,1280356.0,2018-02-22 08:30:22,C,None,None
8,1280355.0,2018-02-22 08:30:21,C,None,None
9,1280354.0,2018-02-22 08:30:20,C,None,None


Bankfile Table

,SORT_CODE,ACC_NUMBER,ACC_TYPE,TRANS_CODE,CS_SORT_CODE,CS_ACC_NUMBER,BGC_REF,AMOUNT,NARRATIVE,CHQ_BGC_REF,BEN_NAME,APPLIED_DATE,LOAD_TYPE,CIS_CASE,RECORD_NO,UNIQUE_ORIGINATORS_REF
0,HDR,None,None,None,None,None,None,None,None,None,None,None,M,None,1.0,None
1,300000,00212156,0,99,None,None,0000,00000012345,CCMS,212156419026,None,19100,M,N,2.0,19061410002212156419026
2,300000,00212156,0,99,None,None,0000,00000023456,CIS,212156419056,None,19101,M,Y,3.0,19061410003212156419056
3,TLR,None,None,None,None,None,None,None,None,None,None,None,M,None,4.0,None
4,None,None,None,None,None,None,None,None,None,None,None,None,M,N,5.0,None
5,None,None,None,None,None,None,None,None,bank190614.1,None,None,None,None,X,NaN,None


...

,BAFI_ID,FILE_DATE,FILE_NAME,FILE_TYPE,INBA_FIN_INST_ID,DATE_CREATED,USER_CREATED,PROCESSED,DATE_MODIFIED,USER_MODIFIED,INBA_ACC_NUMBER
0,99627.0,2019-06-14,bank190614.1-212156,FULL,4052.0,2019-06-18 12:30:02,OPS$BATMAN,None,None,None,1046315.0
1,99626.0,2018-09-25,aaa-180925-212156,FULL,4052.0,2018-09-25 13:11:34,MAYD-A,None,None,None,1046315.0
2,99612.0,2018-02-23,bank180223.5-250694,FULL,4052.0,2018-02-23 08:30:15,OPS$BATMAN,None,None,None,1046315.0
3,99611.0,2018-02-23,bank180223.5-221155,FULL,4052.0,2018-02-23 08:30:15,OPS$BATMAN,None,None,None,1046315.0
4,99610.0,2018-02-23,bank180223.5-212156,FULL,4052.0,2018-02-23 08:30:15,OPS$BATMAN,None,None,None,1046315.0
5,99609.0,2018-02-23,bank180223.4-1048539,FULL,4052.0,2018-02-23 08:30:14,OPS$BATMAN,None,None,None,1048539.0
6,99608.0,2018-02-23,bank180223.2-250694,FULL,4052.0,2018-02-23 08:30:10,OPS$BATMAN,None,None,None,1046315.0


Bafi Det Recs

,BFDR_ID,REF_NUMBER,BEN_NAME,AMOUNT,BAFI_BAFI_ID,DR_CR,DATE_APPLIED,TRANS_CODE,POS_IN_DFILE,DATE_CREATED,...,REM_NAME,MATCH_STATUS,USER_MODIFIED,DATE_MODIFIED,PYMT_PAY_ID,ADJ_AMT,BFDR_BFDR_ID,SUPPRESS_CSP,RECE_RECEIPT_NO,UNIQUE_ORIGINATORS_REF
0,4785546.0,212156419056,NULL BEN NAME,234.56,99627.0,C,2019-04-11,99,3.0,2019-06-18 12:30:02,...,CIS,None,None,None,None,None,None,None,None,19061410003212156419056


CCMS Payments

,CIS_FILE_NAME,CIS_RECORD_NO,COLL_ACCOUNT_NUMBER,CCMS_DEBT_TYPE,BOOK_NUMBER,BOOK_TYPE,PAYMENT_ACC_REQ,ACC_ID,LA_REQ_NO,RECEIPT_DATE,RECEIPT_AMOUNT,DATE_CREATED,USER_CREATED,UNIQUE_ORIGINATORS_REF
0,bank190614.1,2.0,212156.0,Contributions,2.121564e+11,LAR,11302470.0,4642057.0,11302470.0,2019-04-10,123.45,2019-06-18 12:30:03.397889,OPS$BATMAN,19061410002212156419026


## Get Bank File Directory Contents
Accesses server via ssh

In [40]:
display(Markdown(time.strftime("**%H:%M:%S %d/%m/%y**")))
#Bankload file locations
for directory in ["/labprod/var/CIS/extint/bankload/",
                  "/labprod/var/CIS/extint/bankload/ccms",
                  "/labprod/var/CIS/extint/bankload/cwx",
                  "/labprod/var/CIS/extint/bankload/processed"]:
    result = run_ssh_command(ssh_host, ssh_user, ssh_password, "ls -Ftl {}".format(directory))
    display(Markdown("**{}**".format(directory)))
    print result

**14:06:59 18/06/19**

**/labprod/var/CIS/extint/bankload/**

total 49
drwxr-xr-x   3 batman   uniq         212 Jun 18 12:30 ccms/
drwxr-xr-x   2 batman   uniq           8 Jun 18 12:30 cwx/
drwxr-xr-x   4 batman   uniq         208 Jun 18 12:30 mainframe/
drwxr-xr-x   2 batman   uniq         267 Jun 18 12:30 processed/
drwxr-xr-x   2 batman   uniq          13 Jun 18 12:30 ccmsbank/
drwxr-xr-x   2 batman   uniq           8 Feb 16  2018 rejected/
drwxr-xr-x   2 batman   uniq          15 Dec 15  2014 ds_pending/
drwxr-xr-x   2 batman   uniq          15 Nov 21  2014 ds_temp/
drwxr-xr-x   2 batman   uniq           2 Mar 29  2012 safe/



**/labprod/var/CIS/extint/bankload/ccms**

total 455
-rw-r--r--   1 batman   uniq         160 Jun 18 12:30 ccms_dat_190614.1
-rw-r--r--   1 batman   uniq        1881 Jun 18 12:30 ccms_rpt_190614.1
drwxr-xr-x   2 batman   uniq           2 Feb 16  2018 curl-ftp/
-rw-r--r--   1 batman   uniq         530 Feb 16  2018 ccms_rpt_150227.8.emailed
-rw-r--r--   1 batman   uniq        7360 Feb 16  2018 ccms_dat_150227.8.transferred
-rw-r--r--   1 batman   uniq        8320 Feb 16  2018 ccms_dat_150227.7.transferred
-rw-r--r--   1 batman   uniq         530 Feb 16  2018 ccms_rpt_150227.7.emailed
-rw-r--r--   1 batman   uniq         530 Feb 16  2018 ccms_rpt_150227.6.emailed
-rw-r--r--   1 batman   uniq       11680 Feb 16  2018 ccms_dat_150227.6.transferred
-rw-r--r--   1 batman   uniq         530 Feb 16  2018 ccms_rpt_150227.5.emailed
-rw-r--r--   1 batman   uniq       10560 Feb 16  2018 ccms_dat_150227.5.transferred
-rw-r--r--   1 batman   uniq           0 Feb 16  2018 ccms_dat_150227.4.transferred
-rw-r--r--   1 batman   uniq         749 F

**/labprod/var/CIS/extint/bankload/cwx**

total 12
-rw-r--r--   1 batman   uniq         530 Jun 18 12:30 cwx_rpt_190614.1
-rw-r--r--   1 batman   uniq         530 Feb 26  2015 cwx_rpt_150226.2
-rw-r--r--   1 batman   uniq         530 Feb 25  2015 cwx_rpt_250215.3
-rw-r--r--   1 batman   uniq         530 Oct 23  2013 cwx_rpt_130607.2
-rw-r--r--   1 batman   uniq         518 Apr 17  2013 cwx_rpt_130417.2.emailed
-rw-r--r--   1 batman   uniq         518 Apr 17  2013 cwx_rpt_130114.2.emailed



**/labprod/var/CIS/extint/bankload/processed**

total 1209
-rw-r--r--   1 batman   uniq         333 Jun 18 12:30 cc190614.1
-rw-r--r--   1 batman   uniq         222 Jun 18 12:28 bank190614.1
-rw-r--r--   1 batman   uniq        7659 Feb 16  2018 cc150227.8
-rw-r--r--   1 batman   uniq        8547 Feb 16  2018 cc150227.7
-rw-r--r--   1 batman   uniq        8103 Feb 16  2018 cc150227.6
-rw-r--r--   1 batman   uniq        9324 Feb 16  2018 cc150227.5
-rw-r--r--   1 batman   uniq         222 Feb 16  2018 cc150227.4
-rw-r--r--   1 batman   uniq         222 Feb 16  2018 cc150227.3
-rw-r--r--   1 batman   uniq         222 Feb 16  2018 cc150227.2
-rw-r--r--   1 batman   uniq         222 Feb 16  2018 cc150227.1
-rw-r--r--   1 batman   uniq        1100 Feb 16  2018 bank150227.1
-rw-r--r--   1 batman   uniq        2363 Feb 16  2018 bank150227.10
-rw-r--r--   1 batman   uniq         993 Feb 16  2018 bank150227.2
-rw-r--r--   1 batman   uniq        1130 Feb 16  2018 bank150227.3
-rw-r--r--   1 batman   uniq        1404 Feb 16  2018 bank150227.4
-